# Special Methods

Before we talk about the __init__() mehod in python. First we talk about the special method in python. The reason we want to have special methods in our classes is for use to design class that act like the native class that python provided, and that provide seamless interaction with Python. A good example is the for the class to act as a extensions of python.

Within the Python language, there are large number of special mehods names, and they fall into the following categories:
   1. Attribute Access: These special methods implement what we see as object.attribute in an expression, object.attribute on the left-hand side of an assignment, and object.attribute in a del statement
   2. Callables: This special method implements what we see as a function that is applied to arguents, much like the built in len() function
   3. Collections: These special methods implement the numerous features of collections. This involves methods like sequence[index], mapping[key], some_set|other_set
   4. Numbers: These special methods provide arithmetic operators and comparison operators. We can use these methods to expand the domain of numbers that python works with
   5. Context: There are two special mehods we'll use to implement a context manager that works with the with statement
   6. Iterators: There are special methods that define an iterator. This isn't essential since generator functions handle this feature so elegantly. However, we will take a look at how we can design our own iterators
   
## Chapter 1 starts

We will start talking about the most basic special methods == __init__(), this method permits a greate deal of latitude in providing the initial values for an object. In the case of an immutable object, this is the essential definition of the instance, and clarity becomes very important. In Chapter 1, we will review the numerous design alternatives for this method

Before we start talking about the __init__() mehod, we will first take a look the class hierarchy in Python, and start with the super class of all, the Object class

## The implicit superclass - Object

Object is a very simple class definition that does not include anything (almost). It is possible for us to create an Object, however, many of the special methods will not work on this class. In the meantime, all the class we make is a extend of the object class, for example:

In [14]:
class X:
    pass

class X(object):
    pass

print(X.__class__)
print(X.__class__.__base__)

<class 'type'>
<class 'object'>


This class is a sub-class of the super class object. As we can see from the type, a class is an object of the class named type, and the base class of the X object we created is the class named object.

### The base class object __init__() method

The fundamental to the life cycle of an object are its: 1. Creation, 2. Initialization, 3. destruction. Here we will focus on talking about the Initialization. 

The object class has a default implementation of __init__() that amounts to just pass. It is not a must to implement __init__() in our class, if we dont implement this __init__() then no instance variables will be created when the object is created, for some class we might not need __init__(). But most of the time, we will implement __init__() to create instance variable for our custom class

It is possible for us to add attributes to an object that is a subclass of the object. For example: 

In [15]:
class Rectangle:
    def area(self):
        return self.length * self.width
    
r = Rectangle()
r.length = 3
r.width = 4
print( "The area of the Rectangle instance is : {}".format(r.area()))

The area of the Rectangle instance is : 12


The above code is legal in Python, but it causes a lot confusion, thus, it is not considered as a good practice, which should be avoided. Though this could be useful in some cases, it is left to the developer to decide. An __init__() method should make the instance variables explict

### Implementing __init__() in a superclass

When an object is created, python first create an empty object, and then call the __init__() method for that object. The __init__() method generally creates the object's instance variables and performs any other one-time processing

In [58]:
class Card: # superClass Card
    def __init__(self, rank, suit):
        self.suit = suit
        self.rank = rank
        self.hard, self.sort = self._points()
        
class NumberCard(Card): # subClass NumberCard(Card)
    def _points(self):
        return int(self.rank), int(self.rank)

class AceCard(Card):
    def _points(self):
        return 1, 11

class FaceCard(Card):
    def _points(self):
        return 10, 10

In the example above, the __init__() method is in the superclass, so that a common initialization in the superclass, Card, applies to all the three subclasses. And we can also see the polymorphic design here, each sub class has their own implementation of the _points()_ method. All the subclasses have identical signatures, and all of them have the same signiture (a.k.a same attributes and methods). Thus objects of these three subclass can be used interchangeably

In [57]:
cards = [AceCard("A", "Heart"),NumberCard("2", "Heart"),NumberCard("3", "Heart")]

Here we created a list of cards that has different card in it. We enumerated the object into the list by hand. However, in the future, we will use factory function to create a list. And before we do that, we take a look at some other issues

### Using __init__() to create manifest constants

Python has no simple formal mechanism for defining an object as immutable, will talk more aobut this in the future chapter. Here in the example we just assume that the object we are using is immutable

The following is a class that we'll use to build four manifest constants:

In [18]:
class Suit:
    def __init__(self, name, symbol):
        self.name = name
        self.symbol = symbol

Now the following is the domain of "constant" built around this class:

In [19]:
Club, Diamond, Heart, Spade = Suit("Club", "♧"), Suit("Diamond", "♢"), Suit("Heart", "♡"), Suit("Spade", "♤")

And now we can recreat cards by doing this:

In [20]:
cards = [AceCard("A", Heart), NumberCard("2", Spade), NumberCard("5", Club)]

Though here it is not much of a improvement over the original cards list, which the Suit of card is represented by a simple string. But this shows a exmple, in more complex cases, there may be a short list of strategy or state object that can be created like this. This can make the Strategy or State design partterns work efficiently by reusing objects from a small static pool of constants.

The difference between, State pattern and Strategy pattern:
   1. The __State__ pattern deals with __what__(stete or type) an object is (in) -- it encapsulates state dependent behavior
   2. The __Strategy__ pattern deals with __how__ an object performs a certain task -- it encapsulates an algorithum

## Leveraging __init__() via a factory function

Previously we create a deck of card with 3 cards via enumerated 3 cards into the list. Now we can build a complete deck of card via a factory function. And this is a much faster way compare to enumerating all 52 cards. In python there are 2 common approaches to factories as follows:

1. We define a function that creates object of the required classes
2. We define a class that has methods for creating objects. This is the full factory design pattern, as described in the books on design patterns. In languages such as java, a factory class hierarchy is required because the lang does not support standalone functions

In python it is not required to have a class for the factory function, but it is a good idea when there are related factories that are complex. The advantages of a class definitions is that we can inheritant the factory class for reusing the factory function we in the class. If we have a factory class, we can add subclasses to the factory class when extending the target class hierarchy. This will result a polymorphic factory class. The different factory class definitions have the same method signatures and can be used interchangeably.

The factory class defination in python is not that helpful if we use that factory once. Thus, we can simply use functions that have the same signatures. 

__Here is a factory function for the Card subclass :__

In [54]:
def card(rank, suit):
    if rank == 1: return AceCard('A', suit)
    elif 2 <= rank < 11: return NumberCard(str(rank), suit)
    elif 11 <= rank < 14:
        name = {11:'J', 12:'Q', 13:'K'}[rank] # this is the same as aMap["key"]
        return FaceCard(name, suit)
    else:
        raise Exception("Rank out of range")

The function above bulds a Card class from a numeric rank number and a suit object. The above is a encapsulation of the construction issues in a single (factory) function, allowing an application to be built without knowing precisely how the class hierarchy and polymorphic design works.

__Here is a example of how we can build a deck with this factory function:__

In [22]:
deck = [card(rank, suit) 
        for rank in range(1, 14) 
            for suit in (Club, Diamond, Heart, Spade)]

The above loop enumerates all the ranks and suits to create a complete deck of 52 cards

### Faulty factory design and the vague else clause

We should not use a catch-all else clause to do any processing, we only use it to raise an exception. It is important to avoid the vague else clause, __An Example__:

In [23]:
def card2(rank, suit):
    if rank == 1: return Card('A', suit)
    elif 2 <= rank < 11: return NumberCard(str(rank), suit)
    else:
        name = {11:"J", 12:"Q", 13:"K"}[rank]
        return FaceCard(name, suit)

In [24]:
deck2 = [card2(rank, suit) for rank in range(13) for suit in (Club, Diamond, Heart, Spade)]

KeyError: 0

Notice the program got interrupted and an error occur, it is because rank now starts from 0 to 13 instead of 1 to 14. Since when the rank == 0 it is in the else clause, thus a card is trying to be created. Thus the error occur. If we use the old function, it is easier to spot the mistake, it is because of "Rank out of range"

Remeber, only use else clause for processing if the condition is obvious. When in doubt, use an elif

### Simplicity and consistency using elif sequences

There are 2 very common factory design patterns in the card factory function we have:
1. An if-elif sequence
2. A mapping

It is __always__ possible for us to replace a mapping with elif conditions. (However, it is not possible for a map to replace if-elif) Here is a example of the new card factory function with the mapping replaced by the if-elif statement:

In [25]:
def card3(rank, suit):
    if rank == 1: return Card("A", suit)
    elif 2 <= rank < 11: return Card(str(rank), suit)
    elif rank == 11: return FaceCard("J", suit)
    elif rank == 12: return FaceCard("Q", suit)
    elif rank == 13: return FaceCard("K", suit)
    else: raise Exception("Rank out of range")

Now as you can see in the above card factory function, the map has been replace via a if-elif clause, and it will work as the same card() function. This function has the advantage that it is more consistent than the previous card factory function

### Simplicity using mapping and class objects

In some other cases mapping works better than the if-elif clause. For complex cases, if-elif might be the only way to describe the process, however, for the simple cases a mapping solution might be the better choice, in the mean time, it is easier to read

In [26]:
def card4(rank, suit):
    class_ = {1: AceCard, 11: FaceCard, 12: FaceCard, 13: FaceCard}.get(rank, NumberCard)
    # dict.get(key, defult=), here it means, case 1, 11, 12, 13 is like the special case, and all others == Number
    # class_ here is a holder of the class == *card
    return class_(rank, suit)

__class___ explain, the postfix underscore naming convention is used when the most fit name for a variable is occupied by a keyword, thus we add postfixed underscore to our variable name to avoid the rule break

In [27]:
deck2 = [card4(rank, suit) for rank in range(1, 14) for suit in (Club, Diamond, Heart, Spade)]

Basically what happend here is that, in the function called card4, a class object called class\_ is created, this object can be AceCard, FaceCard or NumberCard. Now we have passed in a rank and a suit, rank is a number, while the temp dict is mapped by number as key, while card object as value. This is just like the if condition or a switch satement. the class_ is determined by the number, if the number is == 1, the class_ is assigned as a AceCard, if its 11,12,13 then it is assigned as a FaceCard, else, it is default as a NumberCard. after the class_ object is determined. we return it with class_(rank, suit), while the rank is provided by outer for loop and suit is provide by the inner for loop, just like what we have for the card function. __Notice__ for this function, it is not maping rank 1 == "A" or 11,12,13 == "J","Q","K". This can be avoided by using Two parallel mappings, but it is not recommanded, and an alternative solution is that to have __init__() do the mapping of this part

### Mapping to a tuple of values

Here is a example of this kind of factory function design: __The goal of this is to mantain what we have in card4(), while in the same time, provide a rank to string mapping, a.k.a, 11 ==> "J"__

In [44]:
def card5(rank, suit):
    class_, rank_str = {
        1: (AceCard, 'A'),
        11: (FaceCard, 'J'),
        12: (FaceCard, 'Q'),
        13: (FaceCard, 'K'),
    }.get(rank, (NumberCard, str(rank)))
    return class_(rank_str, suit)

__Explain__, in this card5 factory method, we have both:

1. class_
2. rank_str

need to be assigned, we write special case for AceCard, and FaceCard, mapped via key == rank, while left the default to be the NumberCard. Notice we bound the rank string value with each card type, thus, while we are searching values that has key == rank, we change rank to ranked_str at the same time. At the end we achieve the same outcome as the original card() factory function

### The partial function solution

A partial() function is a function that already has some (not all) of its arguments provided. We'll use the partial() function from the functools library to create a partial of a class with the rank argument, __For Example:__

In [45]:
from functools import partial

def cardPartial(rank, suit):
    part_class = {
        1:  partial(AceCard, 'A'),
        11: partial(FaceCard, 'J'),
        12: partial(FaceCard, 'Q'),
        13: partial(FaceCard, 'K'),
    }.get(rank, partial(NumberCard, str(rank)))
    return part_class(suit)

# partial(func, *args, **kwargs)

__Explain__ basically the partial function creates part of the class, and lastly the function card6 finish off by providing the last needed parameter for the class to finally be created

### Fluent APIs for factories

We might have x.a().b() as an object notation. We can think of it as x(a,b). The x.a() function is a kind of partial() function that is waiting for b(). We can think of this as if it were x(a)(b). So, instead of using partial() to update our own object, we can write partial() function into a fluent factory object. We make the setting of the rank object a fluent method that returns self. Setting the suit object will actually create the Card instance. __Example:__

In [30]:
class CardFactory:
    def rank(self, rank):
        # self.class_ init
        self.class_, self.rank_str = {
            1:(AceCard, 'A'),
            11:(FaceCard, 'J'),
            12:(FaceCard, 'Q'),
            13:(FaceCard, 'K'),
        }.get(rank, (NumberCard, str(rank)))
        return self
    def suit(self, suit):
        return self.class_(self.rank_str, suit)

In [31]:
card8 = CardFactory()
deck8 = [card8.rank(r+1).suit(s) for r in range(13) for s in (Club, Diamond, Heart, Spade)]

The fluent API implementation does not change how the __init__() in the card object works, but it odes changed how the client application creates objects

### Implmenting __init__() in each subclass

Now instead of using a factory functions for creating card object, we can shift the conversion of rank number down to each Card object. And in order to do this, this will requires some common initialization of a superclass as well as subclass-specific initialization. We need to follow the __DRY__(Don't Repeat Yourself) principle to keep the code from getting cloned into each of the subclasses.
__For Example:__

In [32]:
class Card:
    pass
class NumberCard(Card):
    def __init__(self, rank, suit):
        self.suit = suit
        self.rank = str(rank)
        self.hard = self.soft = rank
class AceCard(Card):
    def __init__(self, rank, suit):
        self.suit = suit
        self.rank = "A"
        self.hard, self.soft = 1, 11
class FaceCard(Card):
    def __init__(self, rank, suit):
        self.suit = suit
        self.rank = {
            11:"J",
            12:"Q",
            13:"K",
        }[rank]
        self.hard = self.soft = 10

As you can see, it is polymorphic here, however, there is a lacking of truly common initialization, thus added redundancy to the code, here is a different take on this:

In [33]:
class Card:
    def __init__(self, rank, suit, hard, soft):
        self.rank = rank
        self.suit = suit
        self.hard = hard
        self.soft = soft
class NumberCard(Card):
    def __init__(self, rank, suit):
        super().__init__(str(rank), suit, rank, rank) # hard = soft = rank for NumberCard
class FaceCard(Card):
    def __init__(self, rank, suit):
        super().__init__({11:'J', 12:'Q', 13:'K'}[rank], suit, 10, 10)
class AceCard(Card):
    def __init__(sef, rank, suit):
        super().__init__("A", suit, 1, 11)

Now we can look back at the factory function that create the Cards. We currently modify the original card class, and implemented the __init__() at both the super class and the subclass level. This can make oour factory function become simpler. __For Example__:

In [34]:
def card10(rank, suit):
    if rank == 1: return AceCard(rank, suit)
    elif 2 <= rank < 11: return NumberCard(rank, suit)
    elif 11 <= rank < 14: return FaceCard(rank, suit)
    else:
        raise Exception("Rank out of range")

In [35]:
deck10 = [card10(rank, suit) for rank in range(1,14) for suit in (Club, Diamond, Heart, Spade)]

__Notice__ simplifying the factory function should not be our focus. We can see that we use a rather long __init__() to trade for a slightly simple factory function. This is a common trade off. __It is better to stick with more direct but less programmer-friendly init() methods and push the complexity into factory functions. Since a factory function can be very simple, if you with to wrap and encapsulate the construction complexities__

### Simple composite objects

Composite objects can also be called containers. In the example that this book is using, a deck of individual cards can be a composite object. And it can be represent as a list. The list not only provide a container for the cards, but also can be applied to random.shuffle() and deck.pop(), to achieve huffle and to deal cards. Remember it before start to create a composite object right away, check if any built in class can do the same thing. __Example__:

In [59]:
# note to run this cell, the cell is using the first Card class def, and it is using the first card
# factory function
import random

d = [card(r, s) for r in range(1,14) for s in (Club, Diamond, Heart, Spade)]
random.shuffle(d)
hand = [d.pop(), d.pop()]

To design a collection of objects, we have the following three general design strategies:
1. __Warp__: This design pattern is an existing collection definition. This might be an example of the Facade design pattern
2. __Extend__: This design pattern is an existing collection class. This is ordinary subclass definition
3. __Invent__: This is designed from scratch. We'll look at this in the future

These 3 concept are central to OO design, we must always make this choice when designing a class

### Wrapping a collection class
## - Warp
The following is a wrapper design that contains an internal collection:

In [65]:
# note to run this cell, the cell is using the first Card class def, and it is using the first card
# factory function
class Deck:
    def __init__(self):
        self._cards = [card(r+1,s) for r in range(13) for s in (Club, Spade, Heart, Diamond)]
        random.shuffle(self._cards)
    def pop(self):
        return self._cards.pop()

Now the Deck class is defined, and in the heart of this class is a list object. And the pop method that is in the class is simply a very simple wrapper that is around the pop() from the list obj. With this deck class it is not very easy for us to create a Hand instance with the following:

In [61]:
d = Deck()
hand = [d.pop(), d.pop()]

Generally, a __Facade__ design pattern or wrapper class contains methods that are simply delegated to the underlying implementation class. However, this delegation can become wordy. For a sophisticated collection, we may wind up delegating a large number of methods to the wrapped object.

### Extending a collection class
## - Extend

Another apporach to this is to inherit the list object, with the inheritance, we __do not__ need to rewrite the pop method for the class, since the pop() function is automatically implemented via inheritance, __For Example:__

In [64]:
class Deck2(list):
    def __init__(self):
        super().__init__(card(r+1, s) for r in range(13) for s in (Diamond, Spade, Heart, Club))
        random.shuffle(self)

We leverage the superclass's __init__() method to populate our list object with an initial single deck of cards. Then we shuffle the cards. The pop() method is simply inherited from list and works good here. Other methods inherited from the list also work.

### More requirements and another design

Usually in a casino, people are not just using a deck to dealt cards, instead there is a thing called a shoe, within that shoe there are usually multiple decks of cards. However, not all the deck are used in a shoe, there is also a card called marker card, and it mark the range of the card that can be dealt [0:markercard] that means we can't really just use a list object and warp around it to call it a deck. We need a new class to express the deck with more rules applied to it. __For Example__:

In [66]:
class Deck3(list):
    def __init__(self, decks = 1): # default deck in the shoe is 1
        super().__init__()
        for i in range(decks): # shoe size depends on the deck size
            self.extend(card(r+1, s) for r in range(13) for s in (Club, Diamond, Heart, Spade))
        random.shuffle(self)
        burn = random.randint(1,52*decks) # the random marker card location
        for i in range(burn): self.pop() # deal cards till it hit the marker card

Here in this method, we use the __init__() superclass to build an empty collection. Then, we used self.extend() to append the 52 cards to the shoe, however, we can also use super.extend() here since we did not provide any override to the extend method that we intherited from the super class so it doesn't matter if we called self or super here. In the mean time we can also use this for the __init__()

    super().__init__(card(r+1, s) for r in range(13) for s in (Diamond, Spade, Heart, Club) for d in range(decks))
    
Currrently this class provides a collection of cards instances that we can use to emulate casino blackjack as dealt form a shoe. There is a peculiar ritual in a casino where they reveal the burned card (marker card). If we're going to design a card-counting player strategy, we might want to emulate this nuance too

## Complex composite objects

Here is a example of a Hand object that is suitable for emulating the play strategies, __For Example:__

In [67]:
class Hand:
    def __init__(self, dealer_card):
        self.dealer_card = dealer_card
        self.cards = []
    def hard_total(self):
        return sum(c.hard for c in self.cards)
    def soft_total(self):
        return sum(c.soft for c in self.cards)

Now in order to create an instance of Hand, we can use the following code:

In [69]:
d = Deck()
h = Hand(d.pop())
h.cards.append(d.pop())
h.cards.append(d.pop())

Notice here, since self.cards object is a list, thus the append() method can be applied to the object to operate. In the meantime, the creation processes is quite long, it take multiple steps to initialize the collection. __Now__ what we want to do is to create a fluent interface, however that does not change the big picture, and at the same time the hard_total() and soft_total() here isn't really following the rule of the blackjack. Lastly the append function that is implicit, so that might become annoying, when we want to use serialization etc...

### Complete composite object initialization

Ideally, the __init__() initializer method will create a complete instance of an object. This is a bit more complex when careating a complete instance of a container that contains an internal collection of other objects. It'll be helpful if we can build this composite in a single step. Now we changed the Hand object a little bit, this allows the Hand object to load all the items in one step, and here it means load multiple card to init the Hand object, __For Example:__

In [71]:
class Hand2:
    def __init__(self, dealer_card, *cards):
        self.dealer_card = dealer_card
        self.cards = list(cards)
    def hard_total(self):
        return sum(c.hard for c in self.cards)
    def soft_total(self):
        return sum(c.soft for c in self.cards)

With this new Hand class, we now have the ability to create the Hand object in 2 ways:
1. Create it like the previous one:
    >a. With a initial dealer_card and followed by append()
    
2. Create it all at once, then have the ability to append it too
    >b. for example, Hand2(dealer_card, card, card, card ...)
    
__For Example:__

In [73]:
# we first do the 1st way of creating the Hand object
d = Deck()
p = Hand2(d.pop())
p.cards.append(d.pop()); p.cards.append(d.pop())

# we now do the 2nd way of creating the Hand object
d = Deck()
p = Hand2(d.pop(), d.pop(), d.pop())

Building the class like __Hand2__ instead of __Hand__ is often more helpful and easier to deal with when unit testing. In the mean time some of the serialization techniques from the next part will benefit from a way of building a composite object in a single, simple evalution

## Stateless objects without __init__()

Using an example of a degenerate class that doesn't need an __init__() mehthod. This is a common design pattern for the __strategy object__. The Strategy object is plugged into a Master object to implement an algorithum or decision. It may rely on data in the mater object to operate, the Strategy object may not have any data of its ow. We often design strategy classes to follow the __Flyweight__ design patter: That is to avoid internal storage in the Strategy object. All values are provided to the Strategy as method argument values. __The Strategy object itself can be stateless. It's more a collection of method functions than anything else!__ And we will now build such object and call it the GameStrategy class. __For Example:__

In [74]:
class GameStrategy:
    def insurance(self, hand):
        return False
    def split(self, hand):
        return False
    def double(self, hand):
        return False
    def hit(self, hand):
        return sum(c.hard for c in hand.cards) <= 17

Notice this is a well represented case of the strategy class, there is no data in the class all the data it needed is pass to it via the augment for the method within the class. For this kind of class we just need to build a single instance of this strategy for use by various Player instances as shown in the following code:

In [75]:
dumb = GameStrategy() # and this one instance is enough for the Strategy Class, it can be reused multiple times

We can imagine creating a family of related strategy classes, each one using different rules for the various decisions a player is offered in the blackjack.

## Some additional class definitions

We know that a player has 2 strategies:
1. Betting
2. Playing their hand

Each Player instance has a sequence of interactions with a larger simulation engine. And we will call the larger engine the Table class. The Table class requires the following sequence of events by the Player instance. IMO this table class is more like environment or the big picture on what is happening

1. The player must place an initial bet based on the betting strategy
2. The player will then receive a hand
3. If the hand is splittable, the player must decide to split or not based on the play strategy. This can create additional Hand instance. And In some casinos, the additional hands are also splittable
4. For each Hand instance, the player must decide to hit, double, or stand based on the play strategy
5. The player will then receive payouts, and they must update their betting strategy based on their win or losses

From this, we can already see that this Table class is rather large, and it need to provide multiple API methods to receive a bet, create a Hand object, offer split, resolve each hand, and pay off the bets. This is a large object that tracks the state of play with collection of Players.

In [76]:
class Table:
    def __init__(self):
        self.deck = Deck()
    def place_bet (self, amount):
        print("Bet", amount)
    def get_hand(self):
        try:
            self.hand = Hand2(d.pop(),d.pop(),d.pop())
            self.hole_card = d.pop()
        except(IndexError):
            # out of cards: need to shuffle
            self.deck = Deck()
            return self.get_hand()
        print("Deal", self.hand)
        return self.hand
    def can_insure(self, hand):
        return hand.dealer_card.insure

This table class is used by the player to do a bet, create hand, and determine insure or not, any additional methods can be added if it is a part of the action that a player can do. However, in our table class we use a try except clause to handle the empty of the deck. And this might lead to minaor statistical inaccuracies. A more accurate simulation requires us to developing a deck that reshuffles itself when it is empty.

To properly simulate realistic player, a Player class needs to have betting strategy. The betting strategy is a stateful object that determines the level of the initial bet. The betting strategy is changing based on the previous losing or winning. Ideally we would like to have a family of betting strategy objects. Python has a module with decorators that allows us to create an abstract superclass. An informal approach is to create a Strategy object that come with an execption for methods that must be implemented in the subclass, this is simulating the java's extend. __First an example__:

In [84]:
class BettingStrategy:
    def bet(self):
        raise NotImplementedError("No bet method") # an naive implementation
    def record_win(self):
        pass
    def record_loss(self):
        pass

class Flat(BettingStrategy):
    def bet(self):
        return 1

Here since the basic bet method in the super class raises a exception, thus it must be overrided. However, there is another way of doing this, via the python's abc module

In [110]:
import abc
from abc import abstractmethod

class BettingStrategy2(metaclass=abc.ABCMeta):
    @abstractmethod
    def bet(self):
        return 1
    def record_win(self):
        pass
    def record_loss(self):
        pass

This has the advantage to disallow any instance creation of the class and its subclasses. This force the developer to override the bet() method

## Multi-strategy __init__()

We may have object that are creatd from a variety of souces. For example, we might need to clone an object as part of creating a memento, or freeze an object so that it can be used as the key of a dictionary or place it into a set; this is the idea behind the set and frozenset built-in classes

There are several overall design patterns that have multiple ways to build an object One design patterns is complex __init__() that is called multi-strategy initialization. Also, there are multiple class-level(static) constructor methods. These 2 are incompatible approaches. They have radically different interfaces

Here is an example of turning a mutable Hand object into a frozen, immutable Hand object. The following is an example of a Hand object that can be built in either of the 2 ways:

In [112]:
class Hand3:
    def __init__(self, *args, **kw):
        if len(args) == 1 and isinstance(args[0], Hand3): # case 1
            # Clone an existing hand; often a bad idea
            other = args[0]
            self.dealer_card = other.dealer_card
            self.cards = other.cards
        else: # case 2
            # Build a fresh, new hand
            dealer_card, *cards = args
            self.dealer_card = dealer_card
            self.cards = list(cards)

At the first case, a Hand3 instance has been built from a exisiting Hand3 object. In the second case, a Hand3 object has been built from individual Card instances. This is the way a frozenset object can be built from individual items or an existing set object. We look more at creating immutable objects in the future. Creating a new Hand from an existing Hand allows us to create a memento of Hand object using a construct like the __following__:

In [117]:
h = Hand3(deck.pop(), deck.pop(), deck.pop())
# by providing 3 args, this is using the case 2 of the __init__() in class Hand3
memento = Hand3(h)
# memento is like a snap, it save the old instance of the Hand3, essentially what we did here is like cloning

Above, we save the Hand object in the memento variable. This can be used to compare the final with the original hand that was dealt, or we can freeze it for use in a set or mapping too.
> http://code.activestate.com/recipes/576527-freeze-make-any-object-immutable/

> about freeze()

### More complex initialization alternatives

Sometimes, in order for us to have multi-strategy init, we are often forced to givfe up on specific named parameters. This has its disadvantages since it might force us to use the opaque, meaningless parameter names, however, at the same time it did provide us flexibity. It requires a great deal of documentation explaining the varieant use cases. Here is a example of this apply to the Hand class: __Personally I think this is trying to emulate Java's overload for the constructor__

In [119]:
class Hand4:
    def __init__(self, *args, **kw):
        if len(args) == 1 and isinstance(args[0], Hand4):
            # if it is Hand4 and with len == 1 then: clone
            other = args[0]
            self.dealer_card = other.dealer_card
            self.cards = other.cards
        elif len(args) == 2 and isinstance(args[0], Hand4) and 'split' in kw:
            # split an existing hand
            other, card = args
            self.dealer_card = other.dealer_card
            self.cards = [other.cards[kw['split'], card]]
        elif len(args) == 3:
            # build a fresh new hand
            dealer_card, *cards = args
            self.dealer_card = dealer_card
            self.cards = list(cards)
        else:
            raise TypeError("Invalid constructor args args={0!r} kw = {1!r}".format(args, kw))
    def __str__(self):
        return ",".join(map(str, self.cards))

This design involves getting extra cards to build proper, split hands. When we create the Hand4 object from another Hand4 object, we provide a split keyword argument that uses the index of the Card class from the original Hand4 object. __For Eample:__ The following shows how we'd use this to split a hand:

In [ ]:
d = Deck()
h = Hand4(d.pop(), d.pop(), d.pop())
s1 = Hand4(h, d.pop(), split=0)
s2 = Hand4(h, d.pop(), split=1)

We created an initial h instance of Hand4 and split it into 2 other Hand4 instances, s1 and s2, and dealt an additional Card class into each. The rules of blackjack only allow this when the initial hand has 2 cards or equal rank __//I am not sure about the split concept of blackjack...need to check it out__. The __init__() in this Hand4 class is rather complex, however it provide flexibility to create the object. But also come with the disadvantages that it need lots of docstring to explain all these variations

### Initializing static methods

When we have multiple ways to create an object, it is sometimes more clear to use static methods to create and return instances rather than the comple __init__() methods. __I prefer this way__. Using the static methods as surrogate constructors is a tiny syntax change in construction, but it has huge advantages when organizing the code. __For Example__:

In [121]:
class Hand5:
    def __init__(self, dealer_card, *cards):
        self.dealer_card = dealer_card
        self.cards = list(cards)
    @staticmethod
    def freeze(other):
        hand = Hand5(other.dealer_card, *other.cards)
        return hand
    @staticmethod
    def split(other, card0, card1):
        hand0 = Hand5(other.dealer_card, other.cards[0], card0)
        hand1 = Hand5(other.dealer_card, other.cards[1], card1)
        return hand0, hand1
    def __str__(self):
        return ", ".join(map(str, self.cards))

One method freezes or creates a memento version. The other method splits a Hand5 instance to create 2 new child instance of Hand5. This is piece of code is much more readable and preserves the use of the parameters names to explain the interface, here is how we use this code class:

In [122]:
d = Deck()
h = Hand5(d.pop(), d.pop(), d.pop())
s1, s2 = Hand5.split(h, d.pop(), d.pop())

We created a initial h instance of Hand5, split it into 2 other hands, s1 and s2, and dealt an additional card class into each. The spit() static method is much simpler compare to the Hand4 example. However, it does not follow the pattern of creating a fronzenset object from an existing set object

#### Then the book talked more about the __init__() design tricks. I dont really think it's that important, and finished the chapter by talking about encapsulation in Python, but just a touch of it, with some naming conventions.

# Summary

In this chapther, it talked about the design alternatives of the __init__() method. In the next chapter, more special methods will be talked about